In [ ]:
import numpy as np
import pandas as pd

import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import auc, f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords
import re
from scipy.sparse import  hstack

%matplotlib inline

In [ ]:
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')

In [ ]:
train_df.title = train_df.title.str.lower()
test_df.title = test_df.title.str.lower()

In [ ]:
#обрабатываем url
stop = ['ru', 'org' , 'com', 'by']

def  re_stop(text, mystopwords = stop):
    try:
        return u" ".join([token for token in text.split('-') if not token in mystopwords])
    except:
        return u""
    
train_df.url = train_df.url.apply(re_stop)  
test_df.url = test_df.url.apply(re_stop)

In [ ]:
#обрабатываем url
stop = ['ru', 'org' , 'com', 'by']

def  re_stop1(text, mystopwords = stop):
    try:
        return u" ".join([token for token in text.split('.') if not token in mystopwords])
    except:
        return u""
    
train_df.url = train_df.url.apply(re_stop1)  
test_df.url = test_df.url.apply(re_stop1)

In [ ]:
train_df.title = train_df.title + ' ' + train_df.url
test_df.title = test_df.title + ' ' + test_df.url

In [ ]:
# Оставляем только буквенные символы
regex = re.compile(u"[а-яёa-z]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))

In [ ]:
train_df.loc[:, 'title'] = train_df.title.apply(words_only)
test_df.loc[:, 'title'] = test_df.title.apply(words_only)

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Удаляем русские стоп-слова
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', u'также', 'мл', 'т', 'д', '-', '-']

def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return u" ".join([token for token in text.split() if not token in mystopwords])
    except:
        return u""
    
train_df.title = train_df.title.apply(remove_stopwords)  
test_df.title = test_df.title.apply(remove_stopwords)  

In [ ]:
# Удаляем английские стоп-слова
mystopwords1 = stopwords.words('english')

def  remove_stopwords(text, mystopwords = mystopwords1):
    try:
        return u" ".join([token for token in text.split() if not token in mystopwords])
    except:
        return u""
    
train_df.title = train_df.title.apply(remove_stopwords)  
test_df.title = test_df.title.apply(remove_stopwords) 

In [ ]:
!pip install pymystem3

In [ ]:
!mkdir ./data
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz -O ./data/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf ./data/mystem-3.0-linux3.1-64bit.tar.gz -C ./data

--2020-11-10 21:26:04--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.241, 5.45.205.245, 5.45.205.244, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.241|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskstoredata04.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-11-10 21:26:05--  http://cache-mskstoredata04.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskstoredata04.cdn.yandex.net (cache-mskstoredata04.cdn.yandex.net)... 37.9.96.15, 2a02:6b8:0:3706::13
Connecting to cache-mskstoredata04.cdn.yandex.net (cache-mskstoredata04.cdn.yandex.net)|37.9.96.15|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘./data/mystem-3.0-linux3.1-64bit.tar.gz’

./

In [ ]:
from pymystem3 import Mystem

# нормализуем текст russia
m = Mystem(mystem_bin='./data/mystem')
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

train_df.title = train_df.title.apply(lemmatize)
test_df.title = test_df.title.apply(lemmatize)

In [ ]:
# нормализуем текст english
import nltk 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import 	WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize1(text):
    return " ".join([wordnet_lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)]) 

train_df.title = train_df.title.apply(lemmatize1)
test_df.title = test_df.title.apply(lemmatize1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
train_df.title[143]

'большой шар кукла lol купить краснодар цена руб дата размещение игрушка игра youla'

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
df_train, df_val = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
X_train = df_train["title"].values
y_train = df_train['target'].values

X_val = df_val["title"].values
y_val = df_val['target'].values

X_test = test_df["title"].values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizerC = CountVectorizer()
vectorizerT = TfidfVectorizer(lowercase=True, ngram_range=(2, 4), analyzer='char', min_df=3, max_df=0.1)

In [ ]:
X_train_vectorizedT = vectorizerT.fit_transform(X_train)
X_val_vectorizedT = vectorizerT.transform(X_val)
X_test_vectorizedT = vectorizerT.transform(X_test)

Логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C = 100, class_weight='balanced',solver='liblinear')

In [ ]:
"""parameters = {'C': [0.001,  0.01, 0.1, 1, 10, 100]}"""

"parameters = {'C': [0.001,  0.01, 0.1, 1, 10, 100]}"

In [ ]:
"""grid_search = GridSearchCV(LR, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=1)"""

"grid_search = GridSearchCV(LR, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=1)"

In [ ]:
"""grid_search.fit(X_train_vectorizedT, y_train)"""

'grid_search.fit(X_train_vectorizedT, y_train)'

In [ ]:
"""grid_search.best_params_"""

'grid_search.best_params_'

In [ ]:
LR.fit(X_train_vectorizedT, y_train)
y_LRT = LR.predict(X_val_vectorizedT)
f1_score(y_val, y_LRT)

0.9854376219786819

In [ ]:
l = LR.predict_proba(X_train_vectorizedT)
L = LR.predict_proba(X_val_vectorizedT)

Случайный Лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators = 200, random_state=42)

In [ ]:
RFC.fit(X_train_vectorizedT, y_train)
y_RFCT = RFC.predict(X_val_vectorizedT)
f1_score(y_val, y_RFCT)

0.9787492410443229

In [ ]:
R = RFC.predict_proba(X_val_vectorizedT)
r = RFC.predict_proba(X_train_vectorizedT)

Итог

In [ ]:
LR_stack = LogisticRegression(C = 100, class_weight='balanced',solver='liblinear')
LR_stack.fit(hstack([X_train_vectorizedT, l, r]), y_train.ravel())
f1_score(y_val, LR_stack.predict(hstack([X_val_vectorizedT, L, R])))

0.9852498494882602

In [ ]:
Rt = RFC.predict_proba(X_test_vectorizedT)
Lt = LR.predict_proba(X_test_vectorizedT)

In [ ]:
test_df["target"] = LR_stack.predict(hstack([X_test_vectorizedT, Lt, Rt])).astype(bool)

test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
